<a href="https://colab.research.google.com/github/taxevader007/markovModelUni/blob/main/markovmodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descripción:

 Modelo de prediccion del crudo del petroleo usando cadenas de markov y regresion lineal




# Implementación numérica:

Utilizar un métodos numéricos

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

ticker = "^IXIC" #nasdaq
data = yf.download(ticker, start= "2010-01-01",end= "2024-04-30")


[*********************100%%**********************]  1 of 1 completed


In [ ]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,2294.409912,2311.149902,2294.409912,2308.419922,2308.419922,1931380000
2010-01-05,2307.270020,2313.729980,2295.620117,2308.709961,2308.709961,2367860000
2010-01-06,2307.709961,2314.070068,2295.679932,2301.090088,2301.090088,2253340000
2010-01-07,2298.090088,2301.300049,2285.219971,2300.050049,2300.050049,2270050000
2010-01-08,2292.239990,2317.600098,2290.610107,2317.169922,2317.169922,2145390000
...,...,...,...,...,...,...
2024-04-23,15525.549805,15730.349609,15510.589844,15696.639648,15696.639648,4951670000
2024-04-24,15805.599609,15839.000000,15634.440430,15712.750000,15712.750000,4735870000
2024-04-25,15375.259766,15644.509766,15343.910156,15611.759766,15611.759766,4743640000


se cargan los datos de el activo en ese periodo de tiempo

In [ ]:
data["daily_return"] = data["Adj Close"].pct_change()
#percentage difference yesterday and today

data["state"] = np.where(data["daily_return"] >= 0, "up" , "down")
#where the daily return is up or down

data

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2010-01-04,2294.409912,2311.149902,2294.409912,2308.419922,2308.419922,1931380000,NaN,down
2010-01-05,2307.270020,2313.729980,2295.620117,2308.709961,2308.709961,2367860000,0.000126,up
2010-01-06,2307.709961,2314.070068,2295.679932,2301.090088,2301.090088,2253340000,-0.003300,down
2010-01-07,2298.090088,2301.300049,2285.219971,2300.050049,2300.050049,2270050000,-0.000452,down
2010-01-08,2292.239990,2317.600098,2290.610107,2317.169922,2317.169922,2145390000,0.007443,up
...,...,...,...,...,...,...,...,...
2024-04-23,15525.549805,15730.349609,15510.589844,15696.639648,15696.639648,4951670000,0.015878,up
2024-04-24,15805.599609,15839.000000,15634.440430,15712.750000,15712.750000,4735870000,0.001026,up
2024-04-25,15375.259766,15644.509766,15343.910156,15611.759766,15611.759766,4743640000,-0.006427,down


datos de todos los dias con apertura y cierre y si cerro arriba o abajo con respecto al dia anterior

In [ ]:
# Calculate counts
up_counts = len(data[data["state"] == "up"])
down_counts = len(data[data["state"] == "down"])

# Calculate transition probabilities
up_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "up")]) / len(data.query('state == "up"'))
up_to_down = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down")]) / len(data.query('state == "up"'))
down_to_up = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "up")]) / len(data.query('state == "down"'))
down_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "down")]) / len(data.query('state == "down"'))

# Create transition matrix
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_down],
    "down": [down_to_up, down_to_down]
}, index=["up", "down"])

print(transition_matrix)

            up      down
up    0.550175  0.562773
down  0.449326  0.437227


The matrix represents the transition probabilities between two states: “up” and “down.”


The rows and columns correspond to the current state and the next state, respectively.

In summary, the transition matrix provides insights into how likely the system is to change from one state to another. In this case, the probabilities suggest that transitions from “up” to “down” are more likely than vice versa.

In [ ]:
prob_stay_up = transition_matrix.loc["up", "up"]
prob_stay_down = transition_matrix.loc["down", "down"]

print(f"Probability of staying 'up': {prob_stay_up:.4f}")
print(f"Probability of staying 'down': {prob_stay_down:.4f}")

Probability of staying 'up': 0.5502
Probability of staying 'down': 0.4372


Up to Up ():
If the current state is “up,” there is a % chance that it will remain in the “up” state in the next step.

Down to Down ():
If the current state is “down,” there is a % chance that it will remain in the “down” state in the next step.

In [ ]:
#down_after_five_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down") & (data["state"].shift(-2) == "down")]) / len(data.query('state == "up"'))

#print(f"The probability of a down day after 6 consecutive up days is approximately {down_after_five_up:.4f}")